In [66]:
import utils
import predict
import time as tm
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.datasets import make_classification
import scipy
import pandas as pd
from sklearn.model_selection import train_test_split

In [67]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.utils.data as data_utils
import torch.nn.functional as F

In [68]:
# load Data
dictSize = 225
(X, y) = utils.loadData( "train", dictSize = dictSize )
X = scipy.sparse.csr_matrix.toarray(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [69]:
# https://discuss.pytorch.org/t/multi-class-classification/47565
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.layer1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.relu(out)
        out = self.layer2(out)
        return out
    
    def predict(self, x):
        with torch.no_grad():
            outp = self.forward(x)
            return F.softmax(outp)
# add softmax layer: used for multiclass classification

In [70]:
model = NeuralNet(225 , 1000, 51) 

In [71]:
loss_function = nn.CrossEntropyLoss()
model_opt = optim.SGD(model.parameters(), lr = 0.02)

train_set_X = Variable(torch.from_numpy(X_train)).float()
train_set_y = Variable(torch.LongTensor(y_train)).long()
test_set_X = Variable(torch.from_numpy(X_test)).float()
test_set_y = Variable(torch.LongTensor(y_test)).long()

In [72]:
epochs = 50
for epochs in range(epochs):
    model_opt.zero_grad()
    out = model(train_set_X)
    loss = loss_function(out, train_set_y)
    loss.backward()
    model_opt.step()

In [73]:
ans = model.predict(test_set_X)
# for i in range(3299):
#     ans_max,ind = torch.max(ans[i],0)
#     print (test_set_y[i].numpy() - ind.numpy())
    

/home/aditya/Vpy35/lib/python3.5/site-packages/ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [74]:
k = 5
y_pr = ans.numpy() 
y_pred = np.argsort(-y_pr,axis=1)[:,:k]
print (y_pred)
print (y_test)

[[ 3  2  4  1  7]
 [ 2  1  3  4  9]
 [ 2  3  4  1 10]
 ...
 [ 2  1  3  4  9]
 [ 2  1  4  3  9]
 [ 2  3  4  1 10]]
[3. 2. 3. ... 9. 4. 3.]


In [75]:
# eval functions for Deep Learning
preck = utils.getPrecAtK( y_test, y_pred, k )
# The macro precision code takes a bit longer to execute due to the for loop over labels
mpreck = utils.getMPrecAtK( y_test, y_pred, k )

# According to our definitions, both prec@k and mprec@k should go up as k goes up i.e. for your
# method, prec@i > prec@j if i > j and mprec@i > mprec@j if i > j. See the assignment description
# to convince yourself why this must be the case.

print( "prec@1: %0.3f" % preck[0], "prec@3: %0.3f" % preck[2], "prec@5: %0.3f" % preck[4] )
# Dont be surprised if mprec is small -- it is hard to do well on rare error classes
print( "mprec@1: %0.3e" % mpreck[0], "mprec@3: %0.3e" % mpreck[2], "mprec@5: %0.3e" % mpreck[4] )
print ("prec matrix",preck)
print ("mprec matrix",mpreck)

prec@1: 0.321 prec@3: 0.640 prec@5: 0.756
mprec@1: 1.485e-02 mprec@3: 6.839e-02 mprec@5: 9.979e-02
prec matrix [0.32121212 0.5469697  0.64030303 0.69363636 0.75575758]
mprec matrix [0.01485361 0.05146679 0.06839326 0.08510638 0.09979241]


In [76]:
tr_latent_X = data_utils.TensorDataset(train_set_X, train_set_y)
te_latent_X = data_utils.TensorDataset(test_set_X,  test_set_y)
train_loader_X = torch.utils.data.DataLoader(dataset=tr_latent_X)
test_loader_X = torch.utils.data.DataLoader(dataset=te_latent_X)

In [77]:
def train(epoch):
    model.train()

    train_loss = 0

    for batch_idx, (data,label) in enumerate(train_loader_X):

        model_opt.zero_grad()

        out = model(data)
        loss = loss_function(out, label)

        loss.backward()
        train_loss += loss.item()
        model_opt.step()

        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader_X.dataset),
                100. * batch_idx / len(train_loader_X), loss.item() / len(data)))
    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(train_loader_X.dataset)))

In [ ]:
for epoch in range(1, 10):
    train(epoch)

Train Epoch: 1 [0/6700 (0%)]	Loss: 2.782231
Train Epoch: 1 [100/6700 (1%)]	Loss: 2.185000
Train Epoch: 1 [200/6700 (3%)]	Loss: 0.777501
Train Epoch: 1 [300/6700 (4%)]	Loss: 4.263538
Train Epoch: 1 [400/6700 (6%)]	Loss: 3.963913
Train Epoch: 1 [500/6700 (7%)]	Loss: 1.399235
Train Epoch: 1 [600/6700 (9%)]	Loss: 1.802389
Train Epoch: 1 [700/6700 (10%)]	Loss: 0.861583
Train Epoch: 1 [800/6700 (12%)]	Loss: 2.324794
Train Epoch: 1 [900/6700 (13%)]	Loss: 0.489476
Train Epoch: 1 [1000/6700 (15%)]	Loss: 2.900987
Train Epoch: 1 [1100/6700 (16%)]	Loss: 0.046456
Train Epoch: 1 [1200/6700 (18%)]	Loss: 7.725688
Train Epoch: 1 [1300/6700 (19%)]	Loss: 0.208752
Train Epoch: 1 [1400/6700 (21%)]	Loss: 1.557099
Train Epoch: 1 [1500/6700 (22%)]	Loss: 0.621560
Train Epoch: 1 [1600/6700 (24%)]	Loss: 1.850730
Train Epoch: 1 [1700/6700 (25%)]	Loss: 0.042184
Train Epoch: 1 [1800/6700 (27%)]	Loss: 0.900797
Train Epoch: 1 [1900/6700 (28%)]	Loss: 1.109613
Train Epoch: 1 [2000/6700 (30%)]	Loss: 4.435700
Train Epoch

Train Epoch: 3 [3700/6700 (55%)]	Loss: 0.104322
Train Epoch: 3 [3800/6700 (57%)]	Loss: 0.064507
Train Epoch: 3 [3900/6700 (58%)]	Loss: 0.034301
Train Epoch: 3 [4000/6700 (60%)]	Loss: 1.905348
Train Epoch: 3 [4100/6700 (61%)]	Loss: 0.217452
Train Epoch: 3 [4200/6700 (63%)]	Loss: 0.007071
Train Epoch: 3 [4300/6700 (64%)]	Loss: 1.024406
Train Epoch: 3 [4400/6700 (66%)]	Loss: 0.002218
Train Epoch: 3 [4500/6700 (67%)]	Loss: 2.127766
Train Epoch: 3 [4600/6700 (69%)]	Loss: 1.839814
Train Epoch: 3 [4700/6700 (70%)]	Loss: 0.240608
Train Epoch: 3 [4800/6700 (72%)]	Loss: 2.227814
Train Epoch: 3 [4900/6700 (73%)]	Loss: 0.338265
Train Epoch: 3 [5000/6700 (75%)]	Loss: 2.020960
Train Epoch: 3 [5100/6700 (76%)]	Loss: 0.052181
Train Epoch: 3 [5200/6700 (78%)]	Loss: 1.538784
Train Epoch: 3 [5300/6700 (79%)]	Loss: 0.472497
Train Epoch: 3 [5400/6700 (81%)]	Loss: 0.117076
Train Epoch: 3 [5500/6700 (82%)]	Loss: 2.206626
Train Epoch: 3 [5600/6700 (84%)]	Loss: 2.920035
Train Epoch: 3 [5700/6700 (85%)]	Loss: 2

Train Epoch: 6 [600/6700 (9%)]	Loss: 0.127912
Train Epoch: 6 [700/6700 (10%)]	Loss: 0.546995
Train Epoch: 6 [800/6700 (12%)]	Loss: 0.926126
Train Epoch: 6 [900/6700 (13%)]	Loss: 0.110515
Train Epoch: 6 [1000/6700 (15%)]	Loss: 0.227906
Train Epoch: 6 [1100/6700 (16%)]	Loss: 0.006099
Train Epoch: 6 [1200/6700 (18%)]	Loss: 0.039444
Train Epoch: 6 [1300/6700 (19%)]	Loss: 0.021875
Train Epoch: 6 [1400/6700 (21%)]	Loss: 0.009246
Train Epoch: 6 [1500/6700 (22%)]	Loss: 0.129961
Train Epoch: 6 [1600/6700 (24%)]	Loss: 0.048303
Train Epoch: 6 [1700/6700 (25%)]	Loss: 0.002262
Train Epoch: 6 [1800/6700 (27%)]	Loss: 0.003417
Train Epoch: 6 [1900/6700 (28%)]	Loss: 0.344847
Train Epoch: 6 [2000/6700 (30%)]	Loss: 5.736419
Train Epoch: 6 [2100/6700 (31%)]	Loss: 0.037935
Train Epoch: 6 [2200/6700 (33%)]	Loss: 0.534989
Train Epoch: 6 [2300/6700 (34%)]	Loss: 0.008634
Train Epoch: 6 [2400/6700 (36%)]	Loss: 0.104421
Train Epoch: 6 [2500/6700 (37%)]	Loss: 0.044141
Train Epoch: 6 [2600/6700 (39%)]	Loss: 0.1463